In [2]:
import os
import duckdb
import csv

In [3]:
# Read in reviews
review_path = f'{os.getcwd()}/cleaned_english_reviews.parquet'
db = duckdb.connect(":memory:")
db.sql(f"CREATE TABLE data AS SELECT * FROM read_parquet('{review_path}')")

In [4]:
# Read in game data as needed
game_path = f'{os.path.dirname(os.getcwd())}/data/steam/game.parquet'
db.sql(f"CREATE TABLE game AS SELECT * FROM read_parquet('{game_path}')")

In [5]:
# Read in user data as needed
user_path = f'{os.path.dirname(os.getcwd())}/data/steam/user.parquet'
db.sql(f"CREATE TABLE user AS SELECT * FROM read_parquet('{user_path}')")

In [8]:
db.sql("SELECT app_id, user_id, votes_up, votes_funny, text FROM data ORDER BY votes_up DESC")

┌─────────┬───────────────────┬──────────┬─────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
db.sql("""
   SELECT 
      user_id, SUM(votes_up) AS total, AVG(votes_up) AS avg, 
   FROM 
      data 
   GROUP BY 
      user_id 
   ORDER BY 
      total DESC
   LIMIT 10
""")

In [ ]:
db.sql("SELECT user_id, PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY votes_up DESC) AS p90 FROM data GROUP BY user_id ORDER BY p90 DESC LIMIT 10")

In [ ]:
db.sql("""
    WITH min_rev AS (
        SELECT 
            user_id
        FROM data
        GROUP BY user_id
        HAVING COUNT(1) > 5
    )
    SELECT 
       data.user_id, PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY votes_up DESC) AS p90 
    FROM 
       data
    JOIN 
       min_rev ON data.user_id = min_rev.user_id
    GROUP BY data.user_id 
    ORDER BY p90 DESC 
    LIMIT 10
""")

In [ ]:
db.sql("""
    WITH min_rev AS (
        SELECT 
            user_id
        FROM data
        GROUP BY user_id
        HAVING COUNT(1) > 5
    ), top AS (
        SELECT 
            data.user_id, PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY votes_up DESC) AS p90 
        FROM 
            data
        JOIN 
            min_rev ON data.user_id = min_rev.user_id
        GROUP BY data.user_id 
        ORDER BY p90 DESC 
        LIMIT 10
    )
    
    SELECT 
       user.user_id, user.num_games, user.num_reviews, top.p90 
    FROM 
        user
    JOIN
       top ON user.user_id = top.user_id
""")

In [ ]:
count_votes = db.sql("""
    WITH top AS (
        SELECT votes_up, votes_funny//100 AS votes_funny FROM data ORDER BY votes_up DESC LIMIT 10000
    )
       
    SELECT votes_funny, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY votes_up DESC) AS tot FROM top GROUP BY votes_funny ORDER BY votes_funny
""").fetchall()

output = [({"up": vote[1], "funny": vote[0]}) for vote in count_votes]

# Manually removed the top value
with open("./topVoteUpAgg.csv", "w", encoding="utf8") as result:
    header = ["up", "funny"]
    writer = csv.DictWriter(result, fieldnames=header)
    writer.writeheader()
    writer.writerows(output)


In [ ]:
db.sql("""
   WITH top AS (
      SELECT user_id, votes_up, playtime_at_review FROM data ORDER BY votes_up DESC LIMIT 10000
   )
       
   SELECT 
      top.user_id, user.num_games, top.playtime_at_review, user.num_reviews, votes_up 
   FROM 
      top
   JOIN
      user ON top.user_id = user.user_id
   WHERE 
      user.num_games != 0
      AND playtime_at_review > 5
      AND user.num_reviews > 5
      AND user.num_games > 5
   ORDER BY
      votes_up DESC,
      user.num_games DESC
""")

In [ ]:
user_upvotes = db.sql("""
    SELECT 
       playtime_at_review AS playtime, COUNT(1) AS reviews
    FROM data 
    -- Reduce domain to improve graph clarity
    WHERE playtime != 0 AND playtime < 200 AND votes_up > 10
    GROUP BY playtime
    ORDER BY playtime DESC
""").fetchall()

output = [({"playtime": vote[0], "reviews": vote[1]}) for vote in user_upvotes]

with open("./topPlaytimeCount.csv", "w", encoding="utf8") as result:
    header = ["playtime", "reviews"]
    writer = csv.DictWriter(result, fieldnames=header)
    writer.writeheader()
    writer.writerows(output)

In [ ]:
user_upvotes = db.sql("""
    SELECT 
       user.num_reviews AS group_num_review, AVG(data.votes_up) AS up_votes 
    FROM data 
    JOIN user ON data.user_id = user.user_id 
    -- Reduce domain to improve graph clarity
    WHERE user.num_reviews < 500
    GROUP BY group_num_review
    ORDER BY group_num_review DESC
""").fetchall()

output = [({"num_reviews": vote[0], "up_votes": vote[1]}) for vote in user_upvotes]

with open("./topReviewAgg.csv", "w", encoding="utf8") as result:
    header = ["num_reviews", "up_votes"]
    writer = csv.DictWriter(result, fieldnames=header)
    writer.writeheader()
    writer.writerows(output)

In [ ]:
user_upvotes = db.sql("""
    SELECT 
       user.num_games//10 AS group_num_games, COUNT(1) AS reviews
    FROM data 
    JOIN user ON data.user_id = user.user_id 
    -- Reduce domain to improve graph clarity
    WHERE data.votes_up > 10 AND user.num_games != 0 AND user.num_games < 1000
    GROUP BY group_num_games
    ORDER BY group_num_games DESC
""").fetchall()

output = [({"num_games": vote[0], "reviews": vote[1]}) for vote in user_upvotes]

with open("./topUserCount.csv", "w", encoding="utf8") as result:
    header = ["num_games", "reviews"]
    writer = csv.DictWriter(result, fieldnames=header)
    writer.writeheader()
    writer.writerows(output)

In [84]:
user_upvotes = db.sql("""
    SELECT 
       user.num_games//10 AS group_num_games, AVG(votes_up) AS votes_up
    FROM data 
    JOIN user ON data.user_id = user.user_id 
    -- Reduce domain to improve graph clarity
    WHERE user.num_games != 0 AND user.num_games < 1000
    GROUP BY group_num_games
    ORDER BY group_num_games DESC
""").fetchall()

output = [({"num_games": vote[0], "votes_up": vote[1]}) for vote in user_upvotes]

with open("./topUserAgg.csv", "w", encoding="utf8") as result:
    header = ["num_games", "votes_up"]
    writer = csv.DictWriter(result, fieldnames=header)
    writer.writeheader()
    writer.writerows(output)

In [85]:
user_upvotes = db.sql("""
    SELECT 
       playtime_at_review AS playtime, AVG(votes_up) AS votes_up
    FROM data 
    -- Reduce domain to improve graph clarity
    WHERE playtime != 0 AND playtime < 200
    GROUP BY playtime
    ORDER BY playtime DESC
""").fetchall()

output = [({"playtime": vote[0], "votes_up": vote[1]}) for vote in user_upvotes]

with open("./topPlaytimeAgg.csv", "w", encoding="utf8") as result:
    header = ["playtime", "votes_up"]
    writer = csv.DictWriter(result, fieldnames=header)
    writer.writeheader()
    writer.writerows(output)